## Connecting to Presto

The three mandatory arguments to create a connection are host, port, and user. Other arguments such as source allow to identify the origin of the query. A common use case is to use it to tell which service, tool, or code sent the query.

Let's create a connection:

In [7]:
import prestodb.dbapi as presto

conn = presto.Connection(host="presto", port=8080, user="demo")
cur = conn.cursor()
cur

## SWAPI Catalog

The `swapi-connector` plugin exposes the REST server as a single catalog: `star_wars`.

In [12]:
cur.execute("SHOW catalogs")
cur.fetchall()

[['hive'],
 ['minio'],
 ['mongodb'],
 ['mysql'],
 ['postgres'],
 ['swapi'],
 ['system'],
 ['tcph']]

## Star Wars Schema

Within the `swapi` catalog is a single schema called `star_wars`

In [20]:
cur.execute("SHOW schemas FROM swapi");
cur.fetchall();

## Star Wars Tables

The `star_wars` schema maps SWAPI REST endpoints to tables.

In [18]:
cur.execute("SHOW tables FROM swapi.star_wars")
cur.fetchall()

[['films'], ['people'], ['planets'], ['species'], ['starships'], ['vehicles']]

## Querying

We can now query the Star Wars dataset using SQL.

In [22]:
cur.execute("SELECT * FROM swapi.star_wars.people LIMIT 5")
rows = cur.fetchall()

import pandas as pd
from IPython.display import display

df = pd.DataFrame(rows)
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,1,"[1, 2, 3, 6]",[],"[14, 30]","[12, 22]",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,http://swapi.dev/api/people/1/
1,2,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,1,"[1, 2, 3, 4, 5, 6]",[2],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,http://swapi.dev/api/people/2/
2,3,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,8,"[1, 2, 3, 4, 5, 6]",[2],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,http://swapi.dev/api/people/3/
3,4,Darth Vader,202,136,none,white,yellow,41.9BBY,male,1,"[1, 2, 3, 6]",[],[],[13],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,http://swapi.dev/api/people/4/
4,5,Leia Organa,150,49,brown,light,brown,19BBY,female,2,"[1, 2, 3, 6]",[],[30],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,http://swapi.dev/api/people/5/


## Advanced Querying

Presto allows us to do advanced querying/filtering beyond what is provided by the underlying REST API. In this case, the SWAPI does no support querying/filtering at all, however we can still execute more complex SQL queries on it. For example, if we only want people with blue eyes:

In [24]:
cur.execute("SELECT swapi.star_wars.people.name,swapi.star_wars.people.eye_color FROM swapi.star_wars.people WHERE eye_color='blue' LIMIT 5")
rows = cur.fetchall()

import pandas as pd
from IPython.display import display

df = pd.DataFrame(rows)
display(df)

,0,1
0,Luke Skywalker,blue
1,Owen Lars,blue
2,Beru Whitesun lars,blue
3,Anakin Skywalker,blue
4,Wilhuff Tarkin,blue


## Joins

We can even use joins across the tables the `swapi-connector` exposes. For example, if we want to know the name of each character is from we need data from the `people` table and the `planets` table.

In [25]:
cur.execute("SELECT swapi.star_wars.people.name,swapi.star_wars.planets.name FROM swapi.star_wars.people INNER JOIN swapi.star_wars.planets ON swapi.star_wars.people.homeworld=swapi.star_wars.planets.id LIMIT 5")
rows = cur.fetchall()

import pandas as pd
from IPython.display import display

df = pd.DataFrame(rows)
display(df)

,0,1
0,Luke Skywalker,Tatooine
1,C-3PO,Tatooine
2,R2-D2,Naboo
3,Darth Vader,Tatooine
4,Leia Organa,Alderaan
